In [14]:
import os
from dotenv import load_dotenv

# .env dosyasını yükle
load_dotenv()

# .env dosyasındaki anahtarı al
api_key = os.getenv("GOOGLE_API_KEY")




In [15]:
import os, requests, sys
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
exchangerate_api_key=os.getenv("EXCHANGE_API_KEY")
genai.configure(api_key=api_key)

In [16]:
url = f'https://v6.exchangerate-api.com/v6/{exchangerate_api_key}/latest/USD'
response = requests.get(url)
data = response.json()
usd_to_tl_rate = data['conversion_rates']['TRY']
print(f"USD to TRY rate: {usd_to_tl_rate}")

url = f'https://v6.exchangerate-api.com/v6/{exchangerate_api_key}/latest/TRY'
response = requests.get(url)
data = response.json()
usd_to_tl_rate = data['conversion_rates']['USD']
print(f"TRY to USD rate: {usd_to_tl_rate}")


USD to TRY rate: 34.1324
TRY to USD rate: 0.0293


In [17]:
def delivery_cost(distance:float):
    """Calculates delivery cost based on distance in KMs."""
    return distance*10

def tax(amount:float):
    """Calculates tax based on purchased price."""
    return amount*1.5

In [18]:
functions_to_LLM =[delivery_cost,tax]
functions_selected_byLLM = {"delivery_cost": delivery_cost, "tax":tax}

In [20]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest',tools=functions_to_LLM)
model._tools.to_proto()

[function_declarations {
   name: "delivery_cost"
   description: "Calculates delivery cost based on distance in KMs."
   parameters {
     type_: OBJECT
     properties {
       key: "distance"
       value {
         type_: NUMBER
       }
     }
     required: "distance"
   }
 }
 function_declarations {
   name: "tax"
   description: "Calculates tax based on purchased price."
   parameters {
     type_: OBJECT
     properties {
       key: "amount"
       value {
         type_: NUMBER
       }
     }
     required: "amount"
   }
 }]

In [21]:
chat = model.start_chat()
response = chat.send_message('I bought a tv.')
response.candidates

[content {
  parts {
    text: "Okay, and what is the price of the TV? I need to know the price to calculate the tax. \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [22]:
response = chat.send_message('I bought a tv for 100 Dollar.')
response.candidates

[content {
  parts {
    function_call {
      name: "tax"
      args {
        fields {
          key: "amount"
          value {
            number_value: 100
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
]

In [23]:
#print out each of the funcition calls requested from this single call
for part in response.parts:
    if fn :=part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"function name: '{fn.name}' argument name and value: '{args}'")
    else:
        print("No function Call found")

function name: 'tax' argument name and value: 'amount=100.0'


In [24]:
# call the selected function with the decided argument and value
for part in response.parts:
    if fn :=part.function_call:
        func_name = fn.name
        args = fn.args

        if func_name in functions_selected_byLLM:
            result = functions_selected_byLLM[func_name](**args)
            print(f"Function '{func_name}' called and the result is: {result}")
    else:
        print("No function Call found")

Function 'tax' called and the result is: 150.0


## ATM demo

In [25]:
import os, requests, sys
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
exchangerate_api_key=os.getenv("EXCHANGE_API_KEY")
genai.configure(api_key=api_key)

In [26]:
balance = 5000
def deposit(amount:float):
    """Adds the given amout to the balance."""
    global balance
    if amount>0:
        balance+=amount
        msg=f"Successfully deposited {amount} TL. Your new balance is {balance} TL."
    else:
        msg="Invalid deposti amount. Please enter a positive amount"
    return msg

def wihtdraw(amount:float):
    """Withdraws the given amount from the balance if sufficient funds are available."""
    global balance
    if amount>0 and amount<=balance:
        balance-=amount
        msg=f"Successfully withdrew {amount} TL. Your new balance is {balance} TL."
    elif amount>balance:
        msg="Insufficient funds. Your current balance is {balance} TL."
    else:
        msg="Invalid withdrawal amount. Please enter a positive amount."
    return msg

def get_exchange_rates(from_currency:str,to_currency:str):
    """Display the exchange rates from USD to TRY of from TRY to USD."""
    url = f'https://v6.exchangerate-api.com/v6/{exchangerate_api_key}/latest/{from_currency}'
    response = requests.get(url)
    data = response.json()
    rate = data['conversion_rates'][to_currency]
    msg = f"{from_currency} to {to_currency} rate: {rate}"
    return msg

def exit_from_chat():
    """Prints a goodbye message and ends the interaction."""
    print("Thank you for using our bank services.")
    return "Goodbye!"



In [27]:
functions_to_LLM=[deposit,wihtdraw,get_exchange_rates,exit_from_chat]
functions_selected_by_LLM = {
    "deposit": deposit,
    "withdraw":wihtdraw,
    "get_exchange_rates":get_exchange_rates,
    "exit_from_chat":exit_from_chat
}

Provide a system instruction

In [28]:
system_instruction= """
Your are a multi-lingual bank assistant for an ATM system.
You can speak not only in English but any language the user interacts with you.

The customer's account is denominated in Turkish liras (TRY).

Your goal is to guide customers through the following banking functions:
1. Withdraw money (in TRY).
2. Deposit money (in TRY).  
3. Show exchange rates between USD and TRY (both directions).
4. Exit the chat.  


### Guidelilnes
- Focus only on the four banking operations listed above. Do not offer assistance with anything else or any other banking operations.  
- for each customer interaction, predict which banking functio is needed based on their input and initiate the relevant operation.  
- Always confirm the transaction and the amount with the user before function call.  
- After withdreal or deposti transaction provide  the updated account balance in TRY.  
- Greet the user warmly at the start and guide them politely throughourt the interaction. 
- when the user wishes to end the session, choose the "exit" function to conclude the chat.  

### Language Instructions:
- Primarily communicate in English, but if the customer responds in another language (e.g Turkish), continue the conversation in that language (Turkish).  
- Switch the language according to the language user in the last user respond.
- Before replying anything, check the use languagen and switch the language according to it.  

Remember, your focus is to assist with these specific banking functions.  Make sure to correctly infer the user's intent based on their reponses and offer the appropriate service.  

"""

### helper function

In [29]:
def execute_op(fn):
    if fn :=part.function_call:
        func_name=fn.name
        args = fn.args  

        if func_name in functions_selected_by_LLM:
            result = functions_selected_by_LLM[func_name](**args)
            return result
        else:
            return f"Function {func_name} not found"

In [32]:
def init_chat(system_instruction,functions_to_LLM):
    generation_config = {
        "temperature":0.3,
        "top_p":0.95,
        "top_k":65,
        "max_output_tokens":8192,
    }
    model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest',
                                  generation_config=generation_config,
                                  safety_settings={
                                      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,},
                                      tools =functions_to_LLM,
                                      system_instruction=system_instruction)
    chat = model.start_chat()
    return chat

In [35]:
chat = init_chat(system_instruction,functions_to_LLM)
user_request="Hello"
while True:
    response = chat.send_message(user_request)
    for part in response.parts:
        if fn:=part.function_call:          # LLM decides calling a function
            runction_return = execute_op(fn)
            response_part = genai.protos.Part(
                function_response =genai.protos.FunctionResponse(
                    name=fn.name,
                    response={"result": runction_return}))
            response = chat.send_message(response_part)
            clerk_response=response.text
            user_request=input(clerk_response)
        elif clerk_response:=part.text: # LLM decides replying
            user_request=input(clerk_response)
    if "goodbye!" in clerk_response.lower():
        break



ValueError: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.